##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 보다 나은 ML 엔지니어링을 위한 ML 메타데이터 이용


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/mlmd/mlmd_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/mlmd/mlmd_tutorial.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  
</table>

펭귄을 분류하기 위해 프로덕션 ML 파이프라인을 설정하는 시나리오를 가정합니다. 이 파이프라인은 훈련 데이터를 입력하여 모델을 훈련 및 평가하고, 프로덕션으로 푸시합니다.

그러나 나중에 다른 종류의 펭귄이 포함된 더 큰 데이터세트로 이 모델을 사용하려고 하면 모델이 예상대로 작동하지 않고 종을 잘못 분류하기 시작하는 것을 관찰할 수 있습니다.

이 시점에서 관심사는 다음과 같습니다.

- 사용 가능한 유일한 아티팩트가 프로덕션 중인 모델인 경우 모델을 디버그하는 가장 효율적인 방법은 무엇입니까?
- 모델을 훈련하는 데 어떤 훈련 데이터세트가 사용되었습니까?
- 어떤 훈련 실행이 이 잘못된 모델로 이어졌습니까?
- 모델 평가 결과는 어디에 있습니까?
- 어디에서 디버깅을 시작해야 합니까?

[ML 메타데이터(MLMD)](https://github.com/google/ml-metadata)는 ML 모델과 연결된 메타데이터를 활용하여 이를 포함해 여러 질문에 답하는 데 도움을 주는 라이브러리입니다. 간단히 비유하자면 이 메타데이터를 소프트웨어 개발에 기록하는 것으로 생각하면 됩니다. MLMD를 사용하면 ML 파이프라인의 다양한 구성 요소와 관련된 아티팩트 및 계보를 안정적으로 추적할 수 있습니다.

이 튜토리얼에서는 펭귄을 체질량, 팽대부의 길이와 깊이, 지느러미의 길이를 기준으로 3가지 종으로 분류하는 모델을 만들기 위해 TFX 파이프라인을 설정합니다. 그런 다음 MLMD를 사용하여 파이프라인 구성 요소의 계보를 추적합니다.

## Colab의 TFX 파이프라인

Colab은 프로덕션 환경과 크게 다른 경량 개발 환경입니다. 프로덕션 환경에는 여러 분산 시스템에 걸쳐 데이터 수집, 변환, 모델 훈련, 실행 기록 등과 같은 다양한 파이프라인 구성 요소가 있을 수 있습니다. 이 튜토리얼에서는 오케스트레이션과 메타데이터 스토리지에 상당한 차이가 있다는 점을 알아야 합니다. 이 모두는 Colab 내에서 로컬로 처리됩니다. [여기](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#background)에서 Colab의 TFX에 대해 자세히 알아보세요.


## 설치하기

먼저, 필요한 패키지를 설치 및 가져오고, 경로를 설정하고, 데이터를 다운로드합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 꼭 Colab에서 실행되도록 해야 합니다. 물론 로컬 시스템도 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치 및 가져오기

In [ ]:
 !pip install -q tfx

### 패키지 가져오기

#### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, 위의 셀을 처음 실행할 때 위의 "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 런타임 다시 시작..." 메뉴를 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

In [ ]:
import os
import tempfile
import urllib
import pandas as pd

import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

TFX 및 MLMD 버전을 확인합니다.

In [ ]:
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import ml_metadata as mlmd
print('MLMD version: {}'.format(mlmd.__version__))

## 데이터세트 다운로드하기

이 colab에서는 [Github](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)에서 찾을 수 있는 [Palmer Penguins 데이터세트](https://github.com/allisonhorst/palmerpenguins)를 사용합니다. 불완전한 레코드를 제외하여 데이터세트를 처리하고 `island` 및 `sex` 열을 삭제하고 레이블을 `int32` 로 변환했습니다. 데이터세트에는 체질량, 펭귄 팽대부의 길이와 깊이, 지느러미 길이에 대한 334개의 레코드가 포함되어 있습니다. 이 데이터를 사용하여 펭귄을 세 종 중 하나로 분류합니다.

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "penguins_processed.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

## InteractiveContext 생성하기

이 노트북에서 TFX 구성 요소를 능동적으로 실행하기 위해 `InteractiveContext`를 생성합니다. `InteractiveContext`는 임시 MLMD 데이터베이스 인스턴스와 함께 임시 디렉터리를 사용합니다. `InteractiveContext`에 대한 호출은 Colab 환경 외부에서 작동하지 않습니다.

일반적으로, `Context`에서 유사한 파이프라인 실행을 그룹화하는 것이 좋습니다.

In [ ]:
interactive_context = InteractiveContext()

## TFX 파이프라인 구축하기

TFX 파이프라인은 ML 워크플로의 다양한 측면을 수행하는 여러 요소로 구성됩니다. 이 노트북에서는 `ExampleGen`  , `StatisticsGen`, `SchemaGen` 및 `Trainer` 구성 요소를 생성 및 실행하고 `Evaluator` 및 `Pusher` 구성 요소를 사용하여 훈련된 모델을 평가한 후 푸시합니다.

TFX 파이프라인 구성 요소에 대한 자세한 내용은 [구성 요소 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)을 참조하세요.

참고: 개별 구성 요소를 설정하여 TFX 파이프라인을 구성하려면 많은 상용구 코드가 필요합니다. 이 튜토리얼의 목적상 파이프라인 설정의 모든 코드 라인을 완전히 이해하지 못해도 괜찮습니다. 

### ExampleGen 구성 요소 인스턴스화 및 실행하기

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
interactive_context.run(example_gen)

### StatisticsGen 구성 요소 인스턴스화 및 실행하기

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

### SchemaGen 구성 요소 인스턴스화 및 실행하기

In [ ]:
infer_schema = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
interactive_context.run(infer_schema)

### Trainer 구성 요소 인스턴스화 및 실행하기


In [ ]:
# Define the module file for the Trainer component
trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {trainer_module_file}

# Define the training algorithm for the Trainer module file
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

# Features used for classification - culmen length and depth, flipper length,
# body mass, and species.

_LABEL_KEY = 'species'

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema, batch_size: int) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY), schema).repeat()


def _build_keras_model():
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation='relu')(d)
  d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])
  return model


def run_fn(fn_args: tfx.components.FnArgs):
  schema = schema_pb2.Schema()
  tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema)
  train_dataset = _input_fn(
      fn_args.train_files, fn_args.data_accessor, schema, batch_size=10)
  eval_dataset = _input_fn(
      fn_args.eval_files, fn_args.data_accessor, schema, batch_size=10)
  model = _build_keras_model()
  model.fit(
      train_dataset,
      epochs=int(fn_args.train_steps / 20),
      steps_per_epoch=20,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

`Trainer` 구성 요소를 실행합니다.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(trainer_module_file),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=100),
    eval_args=tfx.proto.EvalArgs(num_steps=50))
interactive_context.run(trainer)

### 모델 평가 및 푸시하기

`Pusher` 구성 요소를 사용하여 모델을 제공 디렉터리로 푸시하기 전에 `Evaluator` 구성 요소를 사용하여 모델을 평가하고 'bless' 처리해야 합니다.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),
                                  'serving_model/penguins_classification')

In [ ]:
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='species', signature_name='serving_default')
    ],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.6})))
        ])
    ],
    slicing_specs=[tfma.SlicingSpec()])

In [ ]:
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    schema=infer_schema.outputs['schema'],
    eval_config=eval_config)
interactive_context.run(evaluator)

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
interactive_context.run(pusher)

TFX 파이프라인을 실행하면 MLMD 데이터베이스가 채워집니다. 다음 섹션에서는 MLMD API를 사용하여 이 데이터베이스에서 메타데이터 정보를 쿼리합니다.

## MLMD 데이터베이스 쿼리하기

MLMD 데이터베이스는 세 가지 유형의 메타데이터를 저장합니다.

- 파이프라인 구성 요소와 연결된 파이프라인 및 계보 정보에 대한 메타데이터
- 파이프라인 실행 중에 생성된 아티팩트에 대한 메타데이터
- 파이프라인 실행에 대한 메타데이터

일반적인 프로덕션 환경 파이프라인은 새 데이터가 도착할 때 여러 모델에 이를 제공합니다. 데이터가 제공된 모델에서 잘못된 결과가 발생하면 MLMD 데이터베이스를 쿼리하여 잘못된 모델을 격리할 수 있습니다. 그런 다음 이러한 모델에 해당하는 파이프라인 구성 요소의 계보를 추적하여 모델을 디버그할 수 있습니다.

MLMD 데이터베이스를 쿼리하기 위해 이전에 정의된 `InteractiveContext`로 메타데이터(MD) 저장소를 설정합니다.

In [ ]:
connection_config = interactive_context.metadata_connection_config
store = mlmd.MetadataStore(connection_config)

# All TFX artifacts are stored in the base directory
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

MD 저장소에서 데이터를 보기 위해 몇 가지 헬퍼 함수를 만듭니다.

In [ ]:
def display_types(types):
  # Helper function to render dataframes for the artifact and execution types
  table = {'id': [], 'name': []}
  for a_type in types:
    table['id'].append(a_type.id)
    table['name'].append(a_type.name)
  return pd.DataFrame(data=table)

In [ ]:
def display_artifacts(store, artifacts):
  # Helper function to render dataframes for the input artifacts
  table = {'artifact id': [], 'type': [], 'uri': []}
  for a in artifacts:
    table['artifact id'].append(a.id)
    artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
    table['type'].append(artifact_type.name)
    table['uri'].append(a.uri.replace(base_dir, './'))
  return pd.DataFrame(data=table)

In [ ]:
def display_properties(store, node):
  # Helper function to render dataframes for artifact and execution properties
  table = {'property': [], 'value': []}
  for k, v in node.properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  for k, v in node.custom_properties.items():
    table['property'].append(k)
    table['value'].append(
        v.string_value if v.HasField('string_value') else v.int_value)
  return pd.DataFrame(data=table)

먼저 MD 저장소에서 저장된 모든 `ArtifactTypes` 목록을 쿼리합니다.

In [ ]:
display_types(store.get_artifact_types())

다음으로, 모든 `PushedModel` 아티팩트를 쿼리합니다.

In [ ]:
pushed_models = store.get_artifacts_by_type("PushedModel")
display_artifacts(store, pushed_models)

MD 저장소에서 최근 푸시된 모델을 쿼리합니다. 이 튜토리얼에는 푸시된 모델이 하나만 있습니다. 

In [ ]:
pushed_model = pushed_models[-1]
display_properties(store, pushed_model)

푸시된 모델을 디버깅하는 첫 단계 중 하나는 어떤 훈련 모델이 푸시되었는지 확인하고 해당 모델을 훈련하는 데 어떤 훈련 데이터가 사용되는지 확인하는 것입니다.

MLMD는 모델 출처를 분석하는 데 사용할 수 있는 출처 그래프를 살펴보기 위한 순회 API를 제공합니다. 

In [ ]:
def get_one_hop_parent_artifacts(store, artifacts):
  # Get a list of artifacts within a 1-hop of the artifacts of interest
  artifact_ids = [artifact.id for artifact in artifacts]
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids(artifact_ids)
      if event.type == mlmd.proto.Event.OUTPUT)
  artifacts_ids = set(
      event.artifact_id
      for event in store.get_events_by_execution_ids(executions_ids)
      if event.type == mlmd.proto.Event.INPUT)
  return [artifact for artifact in store.get_artifacts_by_id(artifacts_ids)]

푸시된 모델의 상위 아티팩트를 쿼리합니다.

In [ ]:
parent_artifacts = get_one_hop_parent_artifacts(store, [pushed_model])
display_artifacts(store, parent_artifacts)

모델의 속성을 쿼리합니다.

In [ ]:
exported_model = parent_artifacts[0]
display_properties(store, exported_model)

모델의 업스트림 아티팩트를 쿼리합니다.

In [ ]:
model_parents = get_one_hop_parent_artifacts(store, [exported_model])
display_artifacts(store, model_parents)

모델 훈련에 사용된 훈련 데이터를 가져옵니다.

In [ ]:
used_data = model_parents[0]
display_properties(store, used_data)

이제 모델 훈련에 사용된 훈련 데이터가 있으므로 데이터베이스를 다시 쿼리하여 훈련 단계(실행)를 찾습니다. 등록된 실행 유형 목록에 대해 MD 저장소를 쿼리합니다.

In [ ]:
display_types(store.get_execution_types())

훈련 단계는 `tfx.components.trainer.component.Trainer`라는 `ExecutionType`입니다. MD 저장소를 탐색하여 푸시된 모델에 해당하는 트레이너 실행을 가져옵니다.

In [ ]:
def find_producer_execution(store, artifact):
  executions_ids = set(
      event.execution_id
      for event in store.get_events_by_artifact_ids([artifact.id])
      if event.type == mlmd.proto.Event.OUTPUT)
  return store.get_executions_by_id(executions_ids)[0]

trainer = find_producer_execution(store, exported_model)
display_properties(store, trainer)

## 요약

이 튜토리얼에서는 MLMD를 활용하여 TFX 파이프라인 구성 요소의 계보를 추적하고 문제를 해결하는 방법을 배웠습니다.

MLMD 사용 방법에 대해 자세히 알아보려면 다음 추가 리소스를 확인하세요.

- [MLMD API 설명서](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd)
- [MLMD 가이드](https://www.tensorflow.org/tfx/guide/mlmd)